Extend a sim run on `gromacs_py`
======================


In [7]:
import sys
import os
import shutil

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle, datetime

## To use `gromacs_py` in a project

In [8]:
from gromacs_py import gmx

## Extend Simulation

Fill in the extended time in nanosec and the path to the tpr file from the previous `gromacs_py` production run. 

In [9]:
extended_time_in_ns = 100
tpr_file = "zeb_hb_complex_sim/sys_prod/prod_zeb_hb_phz_complex.tpr"


dt = 0.002
nsteps = 1000 * extended_time_in_ns / dt

sys_ext = gmx.GmxSys()

#Parallelization
nthreads = int(os.environ.get('PBS_NCPUS', '16'))

#Set Parallelization
sys_ext.nt = nthreads
#sys_ext.ntmpi = 1
sys_ext.gpu_id = '0'

sys_ext.extend_sim(tpr_file=tpr_file, nsteps=nsteps)

print("\n\nProduction extension was successful\n\n")

sys_ext.display()

#Checkpoint after finishing
chkpt_fname='checkpoint.ext_' + datetime.date.today().strftime("%Y%m%d") + ".pycpt"
with open(chkpt_fname, 'wb') as py_cpt:
    pickle.dump(sys_ext, py_cpt)

## Post-Production

### Prepare Production Trajectory for Viewing

### Load from checkpoint

In [ ]:
%%script true
import pickle, datetime
chkpt_fname='checkpoint.ext_20240916.pycpt'

with open(chkpt_fname, 'rb') as py_cpt:
    sys_ext = pickle.load(py_cpt)

#### Center Trajectory

In [ ]:
%%script true
# Center trajectory
sys_ext.center_mol_box(traj=True)

## Final Steps
As in any simulation conducted with periodic boundary conditions, molecules may appear "broken" or may "jump" back and forth across the box. To recenter the protein and rewrap the molecules within the unit cell to recover the desired box shape, invoke trjconv. 

Choose "Protein" for centering and "System" for output.

In [ ]:
%%script true
sys_ext.convert_trj(select='Protein\nSystem', fit='rot+trans', pbc='none')